In [22]:
from elasticsearch import Elasticsearch
from openai import OpenAI
import os
from tqdm.auto import tqdm
import json

In [23]:
import json

with open("../data/llm_chunks.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [24]:
es_client = Elasticsearch("http://localhost:9200")

print(es_client.info())   # should show cluster info
print(es_client.ping())   # may still be False sometimes in 8.x


{'name': 'c20e8fdea842', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'xrLBV0aGRwOwDq0941tjEg', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
True


In [30]:
index_settings = {
    "settings":{
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings":{
        "properties":{
            "chunk_text": {"type": "text"},
            "chunk_title": {"type": "text"},
            "keywords": {"type": "keyword"},
            "video_id": {"type": "keyword"},
            "chunk_id": {"type": "keyword"}
        }
    }

}   
index_name = data[0]["video_id"].lower()

# Delete the index if it exists
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)
    print("The index was deleted")

# Create the index
es_client.indices.create(index=index_name, body=index_settings)
print("The index was created")


The index was deleted
The index was created


In [31]:
type(data[0])

dict

In [32]:
if es_client.indices.exists(index=index_name):
    es_client.delete_by_query(
            index=index_name,
            body={"query": {"match_all": {}}}
        )
for d in tqdm(data):
   es_client.index( index=index_name,document=d)

  0%|          | 0/84 [00:00<?, ?it/s]

In [37]:
def elastic_search(query):
    search_query = {
            "size": 5,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": [
                            "chunk_text^2",
                            "chunk_title^3",
                            "keywords^5"
                            ],
                            "type": "best_fields"
                        }
                    },
           
                }
            }
        }
    response = es_client.search(index=index_name, body=search_query)
    return response

In [38]:
response = elastic_search("How does the text describe the balance between capitalism and socialism in China’s economy?")

In [ ]:
response['hits']['hits']

[{'_index': 'podcast0',
  '_id': '7fBQlZkBeIc9tl9KkfSO',
  '_score': 22.363451,
  '_source': {'chunk_text': 'The following is a conversation with Keyu Jin, an economist at the London School of Economics, specializing in China\'s economy, international macroeconomics, global trade imbalances, and financial policy. She wrote the highly lauded book on China titled "The New China Playbook: Beyond Socialism and Capitalism," that details China\'s economic transformation since 1978 to today. And it dispels a lot of misconceptions about China\'s economy that people in the West have. This is the "Lex Fridman Podcast." To support it, please check out our sponsors in the description and consider subscribing to this channel. And now, dear friends, here\'s Keyu Jin.'}},
 {'_index': 'podcast0',
  '_id': 'J_BQlZkBeIc9tl9Kk_Ux',
  '_score': 21.500687,
  '_source': {'chunk_text': 'So maybe can you speak to the difference, another difference is the long term multi-generational thinking that permeate Chi